# Data Cleaning

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from pathlib import Path
import re
from collections import Counter

# ========= CONFIG =========
BKF_PATH   = "bkf_eflora_species_dedup.csv"   # BKF หลัง dedup
OUT7_PATH  = "output (7).csv"                 # จาก WFO map tools
CAND_PATH  = "candidates (3).csv"             # candidates เพิ่มเติม
OUT_DIR    = "."

# ========= HELPERS =========
def norm(x):
    if pd.isna(x): return ""
    return str(x).replace("\u00A0", " ").strip().lower()

def find_col(cols, candidates):
    """หาคอลัมน์แบบทนทานต่อเคสตัวพิมพ์"""
    cols_list = list(cols)
    # หาแบบตรงชื่อก่อน
    for c in candidates:
        if c in cols_list:
            return c
    # หาแบบ lower-case map
    lower_map = {c.lower(): c for c in cols_list}
    for c in candidates:
        if c.lower() in lower_map:
            return lower_map[c.lower()]
    return None

def nonempty_series_like(df, col):
    """คืน Series ของคอลัมน์ (ถ้าไม่มีให้คืนซีรีส์ว่าง) และแปลง nan/None เป็น '' """
    if col is None or col not in df.columns:
        return pd.Series("", index=df.index)
    return (df[col].astype(str)
                 .str.strip()
                 .replace({"nan": "", "None": "", "NaT": ""}))

def wfo_clean(x):
    """ทำความสะอาด WFO -> รูปแบบ wfo-xxxx (ตัวพิมพ์เล็ก, ตัดอักขระแปลก)"""
    if pd.isna(x): return ""
    s = str(x).strip()
    if s == "" or s.lower() in {"none","nan"}:
        return ""
    s2 = s.lower()
    if not s2.startswith("wfo-"):
        s2 = "wfo-" + s2
    s2 = re.sub(r"[^a-z0-9\-]", "", s2)
    return s2

def prefer(left, right):
    """เลือก left ถ้าไม่ว่าง มิฉะนั้นใช้ right"""
    ls = left.astype(str).str.strip()
    rs = right.astype(str).str.strip()
    return np.where(ls != "", ls, rs)

def resolve_eff(df, colname, suffix):
    """คืนชื่อคอลัมน์ที่มีอยู่จริงหลัง merge (เช็คว่ามีการเติม suffix หรือไม่)"""
    if not colname:
        return None
    if f"{colname}{suffix}" in df.columns:
        return f"{colname}{suffix}"
    return colname if colname in df.columns else None

# ========= LOAD =========
df_bkf = pd.read_csv(BKF_PATH)
df_o7  = pd.read_csv(OUT7_PATH)
df_c   = pd.read_csv(CAND_PATH)

# ========= IDENTIFY KEY COLUMNS =========
bkf_spec = find_col(df_bkf.columns, ["specific_name","Specific_name","specific","name","species"])
bkf_wfo  = find_col(df_bkf.columns, ["wfo_id","wfo","wfo_code"])
bkf_url  = find_col(df_bkf.columns, ["species_url","Species_url","url"])  # optional
bkf_full = find_col(df_bkf.columns, ["wfo_full_name","wfo_name","full_name","accepted_name"])

if bkf_spec is None:
    raise KeyError("ไม่พบคอลัมน์ specific_name ใน BKF")

if bkf_wfo is None:
    # ถ้า BKF ไม่มีคอลัมน์ WFO ให้สร้างว่างไว้
    df_bkf["wfo_id"] = np.nan
    bkf_wfo = "wfo_id"

o7_spec  = find_col(df_o7.columns, ["specific_name","specific","name","query","bkf_specific_name"])
o7_wfo   = find_col(df_o7.columns, ["wfo_id","wfo","wfoCode","WFO_ID"])
o7_full  = find_col(df_o7.columns, ["wfo_full_name","wfo_name","full_name","accepted_name","canonicalName"])

c_spec   = find_col(df_c.columns, ["specific_name","specific","name","query","bkf_specific_name"])
c_wfo    = find_col(df_c.columns, ["wfo_id","wfo","candidate_wfo_id","WFO_ID"])
c_full   = find_col(df_c.columns, ["wfo_full_name","wfo_name","full_name","accepted_name","canonicalName"])

# ========= NORMALIZE KEY & WFO =========
df_bkf["specific_name_clean"] = df_bkf[bkf_spec].apply(norm)
if bkf_url: df_bkf["species_url_clean"] = df_bkf[bkf_url].apply(norm)

if o7_spec: df_o7["specific_name_clean"] = df_o7[o7_spec].apply(norm)
if c_spec:  df_c["specific_name_clean"]  = df_c[c_spec].apply(norm)

# ทำความสะอาด WFO ในทุกตารางที่มี
df_bkf[bkf_wfo] = nonempty_series_like(df_bkf, bkf_wfo).apply(wfo_clean)
if o7_wfo:
    df_o7[o7_wfo] = nonempty_series_like(df_o7, o7_wfo).apply(wfo_clean)
if c_wfo:
    df_c[c_wfo]   = nonempty_series_like(df_c, c_wfo).apply(wfo_clean)

# ========= STEP 0: BASELINE =========
missing_before = df_bkf[bkf_wfo].eq("") | df_bkf[bkf_wfo].isna()
print(f"[BASELINE] Missing WFO (BKF): {int(missing_before.sum())} / {len(df_bkf)}")

# ========= STEP 1: JOIN output(7) ด้วย specific_name_clean =========
o7_use_cols = ["specific_name_clean"]
if o7_wfo:  o7_use_cols.append(o7_wfo)
if o7_full: o7_use_cols.append(o7_full)
df_o7_use = (df_o7[o7_use_cols].drop_duplicates("specific_name_clean")
             if o7_spec else pd.DataFrame(columns=o7_use_cols))

df1 = df_bkf.merge(df_o7_use, on="specific_name_clean", how="left", suffixes=("","_o7"))

# ========= STEP 2: PREP candidates — เก็บเฉพาะชื่อที่มี WFO เดียว =========
if (c_spec is not None) and (c_wfo is not None):
    cg = df_c.dropna(subset=[c_wfo]).copy()
    cg[c_wfo] = cg[c_wfo].astype(str).str.strip()
    uniq_one_key = (
        cg.groupby("specific_name_clean")[c_wfo].nunique()
          .reset_index(name="n").query("n == 1")[["specific_name_clean"]]
    )
    c_use = (
        cg.merge(uniq_one_key, on="specific_name_clean", how="inner")
          .drop_duplicates(subset=["specific_name_clean"])
    )
    c_use_cols = ["specific_name_clean", c_wfo]
    if c_full: c_use_cols.append(c_full)
    c_use = c_use[c_use_cols]
else:
    c_use = pd.DataFrame(columns=["specific_name_clean"])

df2 = df1.merge(c_use, on="specific_name_clean", how="left", suffixes=("","_cand"))

# ========= STEP 2.1: RESOLVE EFFECTIVE NAMES หลัง merge =========
o7_wfo_eff  = resolve_eff(df2, o7_wfo,  "_o7")
o7_full_eff = resolve_eff(df2, o7_full, "_o7")
c_wfo_eff   = resolve_eff(df2, c_wfo,   "_cand")
c_full_eff  = resolve_eff(df2, c_full,  "_cand")

# ========= STEP 3: เลือก wfo_id ตามลำดับความสำคัญ =========
bkf_val = nonempty_series_like(df2, bkf_wfo)
o7_val  = nonempty_series_like(df2, o7_wfo_eff) if o7_wfo_eff else pd.Series("", index=df2.index)
c_val   = nonempty_series_like(df2, c_wfo_eff)  if c_wfo_eff  else pd.Series("", index=df2.index)

bkf_has = bkf_val != ""
o7_has  = o7_val  != ""
c_has   = c_val   != ""

df2["wfo_id_final"] = np.select(
    [bkf_has, (~bkf_has) & o7_has, (~bkf_has) & (~o7_has) & c_has],
    [bkf_val, o7_val, c_val],
    default=""
)

df2["wfo_source"] = np.select(
    [bkf_has, (~bkf_has) & o7_has, (~bkf_has) & (~o7_has) & c_has],
    ["bkf_original","output7_exact","candidates_unique"],
    default="unfilled"
)

# ========= STEP 3.1: FLAG CONFLICTS =========
def diff_nonempty(a, b):
    a = a.astype(str).str.strip(); b = b.astype(str).str.strip()
    return (a != "") & (b != "") & (a != b)

df2["conflict_bkf_vs_o7"]   = diff_nonempty(bkf_val, o7_val)
df2["conflict_bkf_vs_cand"] = diff_nonempty(bkf_val, c_val)
df2["conflict_o7_vs_cand"]  = diff_nonempty(o7_val, c_val)
df2["any_conflict"] = df2[["conflict_bkf_vs_o7","conflict_bkf_vs_cand","conflict_o7_vs_cand"]].any(axis=1)


# ========= STEP 4: wfo_full_name = ใช้เฉพาะจาก output(7) เท่านั้น =========
# เตรียมตารางอ้างอิงจากไฟล์ output(7): wfo_id -> wfo_full_name
# หมายเหตุ: โค้ดก่อนหน้านี้ได้ทำความสะอาด o7_wfo แล้ว (wfo_clean)
if o7_wfo and o7_full:
    # สร้าง lookup โดยอาศัย wfo_id ที่สะอาดแล้ว
    o7_id_name = (
        df_o7[[o7_wfo, o7_full]]
        .dropna(subset=[o7_wfo])
        .copy()
    )
    o7_id_name[o7_wfo]  = o7_id_name[o7_wfo].astype(str).str.strip()
    o7_id_name[o7_full] = o7_id_name[o7_full].astype(str).str.strip()

    # กรณีหนึ่ง wfo_id มีหลายชื่อ ให้เลือกตัวแรก (หรือจะเปลี่ยนเป็น .agg('first') ก็ได้)
    o7_id_name = (
        o7_id_name[o7_id_name[o7_wfo] != ""]
        .drop_duplicates(subset=[o7_wfo], keep="first")
    )

    # ทำ dict สำหรับ map
    o7_map = dict(zip(o7_id_name[o7_wfo], o7_id_name[o7_full]))

    # ตั้งค่า final ชื่อเต็มโดย map จาก wfo_id_final ทุกแถว
    df2["wfo_full_name_final"] = df2["wfo_id_final"].map(o7_map).fillna("")

else:
    # ถ้าไฟล์ output(7) ไม่มีคอลัมน์จำเป็น ให้เป็นค่าว่างไว้ก่อน
    df2["wfo_full_name_final"] = ""

# ========= STEP 5: จัดรูปผลลัพธ์ & ไม่ลบคอลัมน์มาตรฐานผิดพลาด =========
df_out = df2.copy()
df_out["wfo_id"] = df_out["wfo_id_final"].replace({"": np.nan})
df_out["wfo_full_name"] = df_out["wfo_full_name_final"].replace({"": np.nan})

# รายการที่จะลบ (เว้นคอลัมน์ผลลัพธ์มาตรฐานไว้)
drop_cols = []
for col in [
    o7_wfo_eff, o7_full_eff, c_wfo_eff, c_full_eff,
    "wfo_id_final", "wfo_full_name_final",
    "specific_name_clean", "species_url_clean",
    "conflict_bkf_vs_o7","conflict_bkf_vs_cand","conflict_o7_vs_cand","any_conflict"
]:
    if col and (col in df_out.columns) and (col not in ["wfo_id","wfo_full_name"]):
        drop_cols.append(col)

df_out = df_out.drop(columns=drop_cols, errors="ignore")

# ========= STEP 6: SAVE MAIN =========
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)
out_enriched = Path(OUT_DIR, "bkf_wfo_priority_bkf_then_output_then_unique_cand.csv")
df_out.to_csv(out_enriched, index=False)
print(f"Saved enriched: {out_enriched}")

unmatched = df_out[df_out["wfo_id"].isna() | (df_out["wfo_id"].astype(str).str.strip()=="")]
out_unmatched = Path(OUT_DIR, "bkf_wfo_unmatched_after_priority.csv")
unmatched.to_csv(out_unmatched, index=False)
print(f"Unmatched rows: {len(unmatched)}  -> {out_unmatched}")

# ========= STEP 7: AUDIT / DIAGNOSTIC =========
audit_cols = [bkf_spec, bkf_wfo, "specific_name_clean"]
if bkf_full: audit_cols.append(bkf_full)
if o7_wfo_eff:  audit_cols.append(o7_wfo_eff)
if o7_full_eff: audit_cols.append(o7_full_eff)
if c_wfo_eff:   audit_cols.append(c_wfo_eff)
if c_full_eff:  audit_cols.append(c_full_eff)

# unique & keep order
audit_cols = [c for i,c in enumerate(audit_cols) if c and (audit_cols.index(c) == i)]

df_audit = df2[audit_cols + [
    "wfo_id_final","wfo_source",
    "conflict_bkf_vs_o7","conflict_bkf_vs_cand","conflict_o7_vs_cand","any_conflict",
    "wfo_full_name_final"
]].copy()

# เปลี่ยนชื่อหัวคอลัมน์ให้อ่านง่าย
rename_map = {}
if bkf_spec: rename_map[bkf_spec] = "bkf_specific_name"
if bkf_wfo:  rename_map[bkf_wfo]  = "bkf_wfo_id"
if o7_wfo_eff:  rename_map[o7_wfo_eff]  = "o7_wfo_id"
if c_wfo_eff:   rename_map[c_wfo_eff]   = "cand_wfo_id"
if bkf_full:    rename_map[bkf_full]    = "bkf_wfo_full_name"
if o7_full_eff: rename_map[o7_full_eff] = "o7_wfo_full_name"
if c_full_eff:  rename_map[c_full_eff]  = "cand_wfo_full_name"
rename_map["wfo_full_name_final"] = "wfo_full_name_final"

df_audit.rename(columns=rename_map, inplace=True)

out_audit = Path(OUT_DIR, "bkf_wfo_audit_conflicts.csv")
df_audit.to_csv(out_audit, index=False)
print(f"Audit conflicts file: {out_audit}")

# Name -> multi WFO (ควร 0 ถ้า unique จริง)
if "specific_name_clean" in df2.columns:
    dup_name = (df2.assign(_wfo=df2["wfo_id_final"].replace({"": np.nan}))
                   .dropna(subset=["_wfo"])
                   .groupby("specific_name_clean")["_wfo"].nunique()
                   .reset_index().query("_wfo > 1"))
    out_dup_name = Path(OUT_DIR, "audit_name_to_multiple_wfo.csv")
    dup_name.to_csv(out_dup_name, index=False)
    print(f"Name -> multi-WFO list: {out_dup_name}")
else:
    print("Name -> multi-WFO list: NA")

# WFO -> หลายชื่อ (พบบ่อยใน synonym)
wfo_counts = (df2.assign(_wfo=df2["wfo_id_final"].replace({"": np.nan}))
                .dropna(subset=["_wfo"])
                .groupby("_wfo")["specific_name_clean"].nunique()
                .reset_index(name="n_names").sort_values("n_names", ascending=False))
out_wfo_multi = Path(OUT_DIR, "audit_wfo_to_multiple_names.csv")
wfo_counts.to_csv(out_wfo_multi, index=False)
print(f"WFO -> multi-names list: {out_wfo_multi}")

# ========= STEP 8: SUMMARY =========
total = len(df_bkf)
filled = total - len(unmatched)
src_counts = Counter(df2["wfo_source"])
conflicts = int(df2["any_conflict"].sum())

print("\n=== COVERAGE & SOURCE ===")
print(f"Total BKF rows: {total}")
print(f"Filled (any source): {filled}  |  Coverage: {filled/total:.1%}")
for k in ["bkf_original","output7_exact","candidates_unique","unfilled"]:
    print(f"  {k:18s}: {src_counts.get(k,0)}")
print(f"Any conflicts flagged: {conflicts}")

[BASELINE] Missing WFO (BKF): 898 / 7469
Saved enriched: bkf_wfo_priority_bkf_then_output_then_unique_cand.csv
Unmatched rows: 145  -> bkf_wfo_unmatched_after_priority.csv
Audit conflicts file: bkf_wfo_audit_conflicts.csv
Name -> multi-WFO list: audit_name_to_multiple_wfo.csv
WFO -> multi-names list: audit_wfo_to_multiple_names.csv

=== COVERAGE & SOURCE ===
Total BKF rows: 7469
Filled (any source): 7324  |  Coverage: 98.1%
  bkf_original      : 6571
  output7_exact     : 679
  candidates_unique : 74
  unfilled          : 145
Any conflicts flagged: 642


In [2]:
# ดูผลหลัง map ครบ (ก่อน export)
display(df2[["specific_name_clean", "wfo_id_final", "wfo_source", "wfo_full_name_final"]].head())

,specific_name_clean,wfo_id_final,wfo_source,wfo_full_name_final
0,myriophyllum siamense (craib) tardieu,wfo-0000373567,output7_exact,Myriophyllum siamense (Craib) Tardieu
1,myriophyllum tetrandrum roxb.,wfo-0001261381,bkf_original,Myriophyllum tetrandrum Roxb.
2,gonocarpus micranthus,wfo-0000715064,bkf_original,
3,rhizophora apiculata blume,wfo-0001131596,bkf_original,Rhizophora apiculata Blume
4,rhizophora mucronata poir.,wfo-0001131556,output7_exact,Rhizophora mucronata Poir.


In [3]:
# ดูเฉพาะแถวที่มี WFO ID แล้ว แต่ไม่มี full name
missing_full = df2[
    (df2["wfo_full_name_final"].isna() | (df2["wfo_full_name_final"].astype(str).str.strip() == "")) &
    (df2["wfo_id_final"].astype(str).str.strip() != "")
]

# แสดงบางคอลัมน์สำคัญเพื่อเช็ค
cols_show = ["specific_name_clean", "wfo_id_final", "wfo_source", "wfo_full_name_final"]
print(f"Missing full name but has WFO ID: {len(missing_full)} rows")
display(missing_full[cols_show].head(20))

Missing full name but has WFO ID: 1509 rows


,specific_name_clean,wfo_id_final,wfo_source,wfo_full_name_final
2,gonocarpus micranthus,wfo-0000715064,bkf_original,
5,bruguiera gymnorhiza (l.) savigny,wfo-0000572747,candidates_unique,
11,ceriops decandra (griff.) ding hou,wfo-0000597936,candidates_unique,
13,pellacalyx parkinsonii c.e.c.fischer,wfo-0000472091,bkf_original,
20,biophytum umbraculum,wfo-0000565517,bkf_original,
27,brackenridgea elegantissima,wfo-0000570704,bkf_original,
29,campylospermum serratum,wfo-0000705577,bkf_original,
38,eriobotrya bengalensis,wfo-0001017659,bkf_original,
39,rhaphiolepis indica (l.) lindl. ex ker,wfo-0001016273,bkf_original,
41,micromeles cuspidata,wfo-0000996758,bkf_original,


In [4]:
missing_full.to_csv("bkf_missing_fullname_with_wfoid.csv", index=False)
print("Saved -> bkf_missing_fullname_with_wfoid.csv")

Saved -> bkf_missing_fullname_with_wfoid.csv


In [5]:
# เอาเฉพาะที่มาจาก BKF เดิม
missing_full_bkf = missing_full.query('wfo_source == "bkf_original"').copy()
print(missing_full_bkf.shape)
display(missing_full_bkf[cols_show].head(20))

(1435, 29)


,specific_name_clean,wfo_id_final,wfo_source,wfo_full_name_final
2,gonocarpus micranthus,wfo-0000715064,bkf_original,
13,pellacalyx parkinsonii c.e.c.fischer,wfo-0000472091,bkf_original,
20,biophytum umbraculum,wfo-0000565517,bkf_original,
27,brackenridgea elegantissima,wfo-0000570704,bkf_original,
29,campylospermum serratum,wfo-0000705577,bkf_original,
38,eriobotrya bengalensis,wfo-0001017659,bkf_original,
39,rhaphiolepis indica (l.) lindl. ex ker,wfo-0001016273,bkf_original,
41,micromeles cuspidata,wfo-0000996758,bkf_original,
43,pyrus pyrifolia,wfo-0001017269,bkf_original,
45,rubus alpestris blume,wfo-0000990767,bkf_original,


In [6]:
# # -*- coding: utf-8 -*-
# import time, random
# import pandas as pd
# from bs4 import BeautifulSoup

# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.common.exceptions import TimeoutException, WebDriverException

# # ===== CONFIG =====
# BASE_URL = "https://www.worldfloraonline.org/taxon/"
# MAX_ATTEMPTS = 4                # ลองซ้ำต่อรหัสสูงสุดกี่ครั้ง
# WAIT_SEC    = 15                # รอ element สูงสุดต่อหนึ่งครั้ง (วินาที)

# # ===== Chrome options ที่ช่วยเรื่องความเสถียร =====
# chrome_opts = Options()
# chrome_opts.add_argument("--headless=new")             # ถ้าอยากเห็นหน้าจอ ให้คอมเมนต์ออก
# chrome_opts.add_argument("--no-sandbox")
# chrome_opts.add_argument("--disable-dev-shm-usage")
# chrome_opts.add_argument("--disable-gpu")
# chrome_opts.add_argument("--disable-blink-features=AutomationControlled")
# chrome_opts.add_argument("--window-size=1280,900")
# chrome_opts.add_argument("--lang=en-US")
# chrome_opts.add_argument("--disable-features=NetworkService")
# chrome_opts.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X) AppleWebKit/537.36 "
#                          "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_opts)
# driver.set_page_load_timeout(WAIT_SEC + 5)

# def extract_name_from_html(html: str):
#     """ดึงชื่อจาก HTML: <em itemprop=scientificName> + <span itemprop=scientificNameAuthorship>"""
#     soup = BeautifulSoup(html, "lxml")

#     # บางหน้าชื่ออยู่ใน h2#page-title > em.taxonName
#     sci = soup.select_one("em[itemprop='scientificName'].taxonName")
#     if not sci:
#         # กันกรณี class ไม่ตรง / ตำแหน่งต่างไป
#         sci = soup.select_one("em[itemprop='scientificName']") or soup.select_one("em.taxonName")

#     auth = soup.select_one("span[itemprop='scientificNameAuthorship']")
#     if not sci:
#         return None
#     sci_name  = sci.get_text(strip=True)
#     auth_name = auth.get_text(strip=True) if auth else ""
#     full_name = f"{sci_name} {auth_name}".strip()
#     return full_name

# def scrape_one_wfo(wfo_id: str):
#     """เปิดหน้า WFO และดึงชื่อ พร้อม retry/backoff"""
#     url = BASE_URL + str(wfo_id).strip()
#     attempt = 0
#     while attempt < MAX_ATTEMPTS:
#         attempt += 1
#         try:
#             driver.get(url)
#             # รอให้มี em[itemprop=scientificName] โผล่มาใน DOM
#             WebDriverWait(driver, WAIT_SEC).until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, "em[itemprop='scientificName'], em.taxonName"))
#             )
#             # หน่วงสั้นๆ เผื่อ author โผล่ตามมา
#             time.sleep(0.6)
#             full_name = extract_name_from_html(driver.page_source)
#             if full_name and full_name.strip():
#                 return full_name, "ok"
#             else:
#                 # บางที DOM มาไม่ครบ ลองรีเฟรชทีละ attempt
#                 status = "no_scientificName_tag"
#                 raise TimeoutException(status)

#         except (TimeoutException, WebDriverException) as e:
#             # ถ้าเจอปัญหาเน็ต/โหลดไม่ครบ ให้ backoff แล้วลองใหม่
#             backoff = min(2.0 * attempt, 6.0) + random.uniform(0.3, 0.8)
#             # ถ้าเป็น net::ERR_CONNECTION... ให้หน่วงเพิ่มอีกหน่อย
#             msg = str(e)
#             if "ERR_CONNEC" in msg or "ERR_TIMED_OUT" in msg or "ERR_FAILED" in msg:
#                 backoff += 2.0
#             time.sleep(backoff)
#             last_err = f"retry_{attempt}: {e}"
#             continue
#         except Exception as e:
#             return None, f"error: {e}"

#     return None, last_err if 'last_err' in locals() else "unknown_error"

# # ===== เลือก 5 แถวแรกที่ยังไม่มีชื่อ =====
# mask_target = (
#     (missing_full_bkf["wfo_id_final"].astype(str).str.strip() != "") &
#     (missing_full_bkf["wfo_full_name_final"].isna() |
#      (missing_full_bkf["wfo_full_name_final"].astype(str).str.strip() == ""))
# )
# targets = missing_full_bkf[mask_target].head(5).copy()
# targets["scraped_name"] = ""
# targets["scrape_status"] = ""

# # ===== รันจริง =====
# for i, row in targets.iterrows():
#     wfo_id = str(row["wfo_id_final"]).strip()
#     if not wfo_id:
#         targets.at[i, "scrape_status"] = "empty_wfo_id"
#         continue

#     full_name, status = scrape_one_wfo(wfo_id)
#     targets.at[i, "scraped_name"] = full_name if full_name else ""
#     targets.at[i, "scrape_status"] = status

#     # พักสั้น ๆ กันโดน rate-limit
#     time.sleep(random.uniform(1.0, 2.0))

# # ปิดเบราว์เซอร์
# driver.quit()

# # ===== อัปเดตกลับเข้า missing_full_bkf =====
# ok_rows = targets[targets["scrape_status"] == "ok"]
# for _, r in ok_rows.iterrows():
#     idx = missing_full_bkf.index[missing_full_bkf["wfo_id_final"].astype(str).str.strip() == r["wfo_id_final"]]
#     missing_full_bkf.loc[idx, "wfo_full_name_final"] = r["scraped_name"]

# print("✅ Done (5 rows)")
# print(targets[["wfo_id_final", "scraped_name", "scrape_status"]])

In [7]:
# -*- coding: utf-8 -*-
import time, random
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, WebDriverException

# ===== CONFIG =====
BASE_URL = "https://www.worldfloraonline.org/taxon/"
MAX_ATTEMPTS = 4           # retry ต่อรหัส
WAIT_SEC    = 15           # รอโหลด element
SAVE_EVERY  = 10           # เซฟผลทุก 25 แถว

# ===== Chrome Options =====
chrome_opts = Options()
chrome_opts.add_argument("--headless=new")  # ถ้าอยากเห็นหน้าจอจริง comment บรรทัดนี้ออก
chrome_opts.add_argument("--no-sandbox")
chrome_opts.add_argument("--disable-dev-shm-usage")
chrome_opts.add_argument("--disable-gpu")
chrome_opts.add_argument("--disable-blink-features=AutomationControlled")
chrome_opts.add_argument("--window-size=1280,900")
chrome_opts.add_argument("--lang=en-US")
chrome_opts.add_argument("--disable-features=NetworkService")
chrome_opts.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X) AppleWebKit/537.36 "
                         "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_opts)
driver.set_page_load_timeout(WAIT_SEC + 5)

# ===== Helper =====
def extract_name_from_html(html: str):
    soup = BeautifulSoup(html, "lxml")
    sci = soup.select_one("em[itemprop='scientificName'].taxonName")
    if not sci:
        sci = soup.select_one("em[itemprop='scientificName']") or soup.select_one("em.taxonName")
    auth = soup.select_one("span[itemprop='scientificNameAuthorship']")
    if not sci:
        return None
    sci_name = sci.get_text(strip=True)
    auth_name = auth.get_text(strip=True) if auth else ""
    return f"{sci_name} {auth_name}".strip()

def scrape_one_wfo(wfo_id: str):
    url = BASE_URL + str(wfo_id).strip()
    attempt = 0
    while attempt < MAX_ATTEMPTS:
        attempt += 1
        try:
            driver.get(url)
            WebDriverWait(driver, WAIT_SEC).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "em[itemprop='scientificName'], em.taxonName"))
            )
            time.sleep(0.8)
            full_name = extract_name_from_html(driver.page_source)
            if full_name:
                return full_name, "ok"
            else:
                raise TimeoutException("no_scientificName_tag")
        except (TimeoutException, WebDriverException) as e:
            backoff = min(2.0 * attempt, 6.0) + random.uniform(0.5, 1.5)
            msg = str(e)
            if "ERR_CONNEC" in msg or "ERR_TIMED_OUT" in msg or "ERR_FAILED" in msg:
                backoff += 2.0
            time.sleep(backoff)
            last_err = f"retry_{attempt}: {e}"
            continue
        except Exception as e:
            return None, f"error: {e}"
    return None, last_err if 'last_err' in locals() else "unknown_error"

# ===== เตรียม dataframe =====
mask_target = (
    (missing_full_bkf["wfo_id_final"].astype(str).str.strip() != "") &
    (missing_full_bkf["wfo_full_name_final"].isna() |
     (missing_full_bkf["wfo_full_name_final"].astype(str).str.strip() == ""))
)
targets = missing_full_bkf[mask_target].copy()
targets["scraped_name"] = ""
targets["scrape_status"] = ""

print(f"🪴 Total to scrape: {len(targets)} rows")

# ===== รันจริงทั้งหมด =====
for i, (idx, row) in enumerate(targets.iterrows(), start=1):
    wfo_id = str(row["wfo_id_final"]).strip()
    if not wfo_id:
        targets.at[idx, "scrape_status"] = "empty_wfo_id"
        continue

    full_name, status = scrape_one_wfo(wfo_id)
    targets.at[idx, "scraped_name"] = full_name if full_name else ""
    targets.at[idx, "scrape_status"] = status

    if status == "ok" and full_name:
        missing_full_bkf.loc[missing_full_bkf["wfo_id_final"] == wfo_id, "wfo_full_name_final"] = full_name

    # พักระหว่างโหลดหน้า (สุ่มเล็กน้อย)
    time.sleep(random.uniform(1.0, 2.2))

    # บันทึกชั่วคราวเป็นระยะ
    if i % SAVE_EVERY == 0:
        temp_out = f"bkf_scrape_progress_{i}.csv"
        targets.to_csv(temp_out, index=False)
        print(f"💾 Progress saved ({i}/{len(targets)}) -> {temp_out}")

# ===== ปิด browser =====
driver.quit()

# ===== บันทึกผลสุดท้าย =====
targets.to_csv("bkf_wfo_scrape_all_results.csv", index=False)
missing_full_bkf.to_csv("bkf_full_after_scrape.csv", index=False)

print("✅ DONE scraping all missing_full_bkf")
print(targets["scrape_status"].value_counts())
print("Saved -> bkf_full_after_scrape.csv")

/Users/thanwaratkeratipasuwat/Desktop/dsi314/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


🪴 Total to scrape: 1435 rows
💾 Progress saved (10/1435) -> bkf_scrape_progress_10.csv
💾 Progress saved (20/1435) -> bkf_scrape_progress_20.csv
💾 Progress saved (30/1435) -> bkf_scrape_progress_30.csv
💾 Progress saved (40/1435) -> bkf_scrape_progress_40.csv
💾 Progress saved (50/1435) -> bkf_scrape_progress_50.csv
💾 Progress saved (60/1435) -> bkf_scrape_progress_60.csv
💾 Progress saved (70/1435) -> bkf_scrape_progress_70.csv
💾 Progress saved (80/1435) -> bkf_scrape_progress_80.csv
💾 Progress saved (90/1435) -> bkf_scrape_progress_90.csv
💾 Progress saved (100/1435) -> bkf_scrape_progress_100.csv
💾 Progress saved (110/1435) -> bkf_scrape_progress_110.csv
💾 Progress saved (120/1435) -> bkf_scrape_progress_120.csv
💾 Progress saved (130/1435) -> bkf_scrape_progress_130.csv
💾 Progress saved (140/1435) -> bkf_scrape_progress_140.csv
💾 Progress saved (150/1435) -> bkf_scrape_progress_150.csv
💾 Progress saved (160/1435) -> bkf_scrape_progress_160.csv
💾 Progress saved (170/1435) -> bkf_scrape_pro